In [ ]:
# https://colab.research.google.com/drive/1rMY0TK5bprEYrxHy6Ao6Gr4t8-LCUthW
!gdown --id 1e1uIwcJ1-MviSn9yk_ldPGffDWVp6yK_

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1e1uIwcJ1-MviSn9yk_ldPGffDWVp6yK_
To: /content/twitter_sentiment_analysis_3cls_dataset.zip
100% 7.97M/7.97M [00:00<00:00, 90.5MB/s]


In [ ]:
!unzip /content/twitter_sentiment_analysis_3cls_dataset.zip

Archive:  /content/twitter_sentiment_analysis_3cls_dataset.zip
  inflating: Twitter_Data.csv        


In [ ]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt
import re
import nltk
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset_path = 'Twitter_Data.csv'
df = pd.read_csv (
    dataset_path
)
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   clean_text  162976 non-null  object 
 1   category    162973 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [ ]:
df = df.dropna ()

In [ ]:
def text_normalize ( text ):
    # Lowercasing
    text = text . lower ()

    # Retweet old acronym "RT" removal
    text = re.sub (r'^rt [\s]+','', text )

    # Hyperlinks removal
    text = re.sub (r'https ?:\/\/.*[\ r\n]*','', text )

    # Punctuation removal
    text = re.sub (r'[^\w\s]','', text )

    # Remove stopwords
    stop_words = set( stopwords . words ('english'))
    words = text . split ()
    words = [ word for word in words if word not in stop_words ]
    text =' '. join ( words )

    # Stemming
    stemmer = SnowballStemmer ('english')
    words = text.split ()
    words = [ stemmer . stem ( word ) for word in words ]
    text =' '. join ( words )
    return text

In [ ]:
vectorizer = TfidfVectorizer(max_features =2000)
X = vectorizer . fit_transform(df['clean_text']).toarray ()

In [ ]:
intercept = np. ones ((
    X. shape [0] , 1)
)
X_b = np. concatenate (
    ( intercept , X),
    axis =1
)

In [ ]:
n_classes = df['category']. nunique ()
n_samples = df['category'].size
y = df['category'].to_numpy () + 1
y = y.astype(np. uint8 )
y_encoded = np. array (
    [np. zeros ( n_classes ) for _ in range ( n_samples )]
)
y_encoded [np. arange ( n_samples ), y] = 1

In [ ]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train , X_val , y_train , y_val = train_test_split (
    X_b , y_encoded ,
    test_size = val_size ,
    random_state = random_state ,
    shuffle = is_shuffle
)

X_train , X_test , y_train , y_test = train_test_split (
    X_train , y_train ,
    test_size = test_size ,
    random_state = random_state ,
    shuffle = is_shuffle
)

In [ ]:
def softmax (z):
    exp_z = np. exp(z)
    return exp_z / exp_z .sum( axis =1) [:, None ]

def compute_loss (y_hat , y):
    n = y. size
    return (-1 / n) * np.sum(y * np.log( y_hat ))

def predict (X, theta ):
    z = np.dot(X, theta )
    y_hat = softmax (z)

    return y_hat

def compute_gradient (X, y, y_hat ):
    n = y. size
    return np.dot(X.T, ( y_hat - y)) / n

def update_theta (theta , gradient , lr):
    return theta - lr * gradient

def compute_accuracy (X, y, theta ):
    y_hat = predict (X, theta )
    acc = (np. argmax (y_hat , axis =1) == np. argmax (y, axis =1)). mean ()
    return acc

In [ ]:
lr = 0.1
epochs = 200
batch_size = X_train . shape [0]
n_features = X_train . shape [1]

np. random . seed ( random_state )
theta = np. random . uniform (
    size =( n_features , n_classes )
)